# TEST MODE

In [1]:
# 30 min + 30 + 10 + 

In [2]:
TEST_MODE=False

# Сиды

In [3]:
import numpy as np
import random
import os

seed=105

os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)


# Импорты

In [4]:
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from catboost import CatBoostClassifier, Pool, CatBoostRegressor
from transformers import AutoModelForSequenceClassification, AutoTokenizer


# Загрузка данных

In [5]:
train_stat=pd.read_parquet('/kaggle/input/attention-on-screen/video_stat (1).parquet')
train=pd.read_parquet('/kaggle/input/attention-on-screen/train (6).parquet')
test=pd.read_parquet('/kaggle/input/attention-on-screen/test (1).parquet')
sample=pd.read_csv('/kaggle/input/attention-on-screen/sample_submission (14).csv')


In [6]:
num_feats=train.select_dtypes(include='number').columns.tolist()

cat_feats=train.select_dtypes(include='object').columns.tolist()

In [7]:
num2_feats=train_stat.select_dtypes(include='number').columns.tolist()

In [8]:
if TEST_MODE:
    train=train[:100]
    test=test[:100]

# EDA

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14154564 entries, 0 to 14154563
Data columns (total 6 columns):
 #   Column           Dtype                        
---  ------           -----                        
 0   event_timestamp  datetime64[ms, Europe/Moscow]
 1   user_id          object                       
 2   region           object                       
 3   city             object                       
 4   video_id         object                       
 5   watchtime        int64                        
dtypes: datetime64[ms, Europe/Moscow](1), int64(1), object(4)
memory usage: 647.9+ MB


# Joining stuff

In [10]:
train_full=train.merge(train_stat, on=['video_id'], how='left')

test_full=test.merge(train_stat, on=['video_id'], how='left')

In [11]:
#train_full.to_csv('for_analysis.csv', index=False)

# Still EDA

In [12]:
#v_duration                               0.004272
#v_likes                                  0.001689

In [13]:
import pandas as pd
import numpy as np

# 1) Keep only train rows whose user is present in test
users_in_test = pd.Index(test['user_id'].dropna().unique())
train_new = train[train['user_id'].isin(users_in_test)].copy()

print(f"train rows:     {len(train):,}")
print(f"train_new rows: {len(train_new):,} ({len(train_new)/len(train):.2%} of train)\n")

# 2) Helper to report both unique-level and row-level coverage
def coverage_stats(train_series: pd.Series, test_series: pd.Series, name: str):
    tr_u = pd.Index(train_series.dropna().unique())
    te_u = pd.Index(test_series.dropna().unique())
    n_common_u = tr_u.intersection(te_u).size
    pct_common_u = n_common_u / te_u.size if te_u.size else np.nan

    # Row-level: how many test rows have a value that exists in train_new
    covered_rows = test_series.isin(tr_u).sum()
    pct_rows = covered_rows / len(test_series)
    unseen_rows = len(test_series) - covered_rows

    print(f"{name}:")
    print(f"  test unique:          {te_u.size:,}")
    print(f"  common unique:        {n_common_u:,}  ({pct_common_u:.2%})")
    print(f"  test rows covered:    {covered_rows:,}  ({pct_rows:.2%})")
    print(f"  unseen test rows:     {unseen_rows:,}  ({1 - pct_rows:.2%})\n")

# 3) Re-check coverage after filtering train -> train_new
coverage_stats(train_new['event_timestamp'], test['event_timestamp'], "event_timestamp")
coverage_stats(train_new['video_id'],         test['video_id'],         "video_id")


train rows:     14,154,564
train_new rows: 3,317,176 (23.44% of train)

event_timestamp:
  test unique:          85,061
  common unique:        85,007  (99.94%)
  test rows covered:    1,333,590  (99.96%)
  unseen test rows:     542  (0.04%)

video_id:
  test unique:          155,006
  common unique:        130,234  (84.02%)
  test rows covered:    1,303,675  (97.72%)
  unseen test rows:     30,457  (2.28%)



# Feature engineering

In [14]:
train_full_new=train_new.merge(train_stat, on=['video_id'], how='left')

In [15]:
useless=['v_cr_click_dislike_7_days','v_cr_click_dislike_30_days',
         'v_avg_watchtime_7_day','v_avg_watchtime_30_day',
         'v_frac_avg_watchtime_7_day_duration','v_frac_avg_watchtime_30_day_duration',
         'v_month_views','v_week_views',
         'v_cr_click_like_7_days','v_cr_click_like_30_days',
         'v_cr_click_comment_7_days','v_cr_click_comment_30_days',
         'v_cr_click_long_view_7_days','v_cr_click_long_view_30_days',
         'v_category_popularity_percent_30_days',
         'v_long_views_7_days','v_long_views_30_days',
         'v_is_deleted','v_is_hidden','row_number'
         
        # 'user_id','category_id','author_id', 'video_id',
         #'description','title',
        # 'region','city'
        ]

In [16]:
train_full_new1=train_full_new.drop(columns=useless)

test_full_new1=test_full.drop(columns=useless)

In [17]:
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd

def make_long_watch_flag(df, duration_col='v_duration', watch_col='watchtime', strict=True, unit='s'):
    # 1) to seconds (if needed) and numeric
    dur = pd.to_numeric(df[duration_col], errors='coerce')
    wt  = pd.to_numeric(df[watch_col],  errors='coerce')
    if unit == 'ms':
        dur = dur / 1000.0
        wt  = wt  / 1000.0

    # 2) thresholds
    thr = np.where(dur > 300.0, dur * 0.25, 30.0)  # 300s = 5 min

    # 3) validity mask and comparison
    valid = dur.notna() & wt.notna() & (dur > 0) & (wt >= 0)
    cmp = (wt > thr) if strict else (wt >= thr)

    # 4) build nullable Int8 target safely (pandas Series + pd.NA)
    y = pd.Series(cmp, index=df.index)
    y.loc[~valid] = pd.NA
    df['y_long'] = y.astype('Int8')  # 0/1 with NA

    # optional: keep threshold for debugging
    df['longwatch_threshold_sec'] = pd.Series(np.where(valid, thr, np.nan), index=df.index)

    return df


# Example:
train_full_new1 = make_long_watch_flag(train_full_new1, 'v_duration', 'watchtime', strict=True)
# If test has watchtime and you want the same flag there:
# test = make_long_watch_flag(test, 'v_duration', 'watchtime', strict=True)

# Quick sanity check
print(train_full_new1['y_long'].value_counts(dropna=False))


/tmp/ipykernel_37/1262398379.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  y.loc[~valid] = pd.NA


y_long
1       1713484
0       1549869
<NA>      53823
Name: count, dtype: Int64


In [18]:
# Вариант 1 (рекомендую)
before = len(train_full_new1)
train_full_new1 = train_full_new1.dropna(subset=['y_long']).copy()
train_full_new1['y_long'] = train_full_new1['y_long'].astype('int8')
print(f"Dropped {before - len(train_full_new1)} rows")


Dropped 53823 rows


In [19]:
'''
Совпадают значения 

v_frac_avg_watchtime_1_day_duration      401816
v_frac_avg_watchtime_7_day_duration      401816
v_frac_avg_watchtime_30_day_duration     401816


v_month_views                              3284
v_week_views                               3284
v_day_views                                3284



v_long_views_1_days                        2402
v_long_views_7_days                        2402
v_long_views_30_days                       2402


'''

'\nСовпадают значения \n\nv_frac_avg_watchtime_1_day_duration      401816\nv_frac_avg_watchtime_7_day_duration      401816\nv_frac_avg_watchtime_30_day_duration     401816\n\n\nv_month_views                              3284\nv_week_views                               3284\nv_day_views                                3284\n\n\n\nv_long_views_1_days                        2402\nv_long_views_7_days                        2402\nv_long_views_30_days                       2402\n\n\n'

In [20]:
'''
v_cr_click_dislike_7_days                   980
v_cr_click_dislike_1_days                   980
v_cr_click_dislike_30_days                  980


v_avg_watchtime_1_day                    220489
v_avg_watchtime_7_day                    220489
v_avg_watchtime_30_day                   220489


v_frac_avg_watchtime_1_day_duration      401816
v_frac_avg_watchtime_7_day_duration      401816
v_frac_avg_watchtime_30_day_duration     401816


v_month_views                              3284
v_week_views                               3284
v_day_views                                3284


v_cr_click_like_1_days                     4869
v_cr_click_like_30_days                    4869
v_cr_click_like_7_days                     4869


v_cr_click_comment_7_days                  1694
v_cr_click_comment_30_days                 1694
v_cr_click_comment_1_days                  1694


v_cr_click_long_view_30_days              28910
v_cr_click_long_view_1_days               28910
v_cr_click_long_view_7_days               28910


v_category_popularity_percent_7_days         56
v_category_popularity_percent_30_days        56

v_long_views_1_days                        2402
v_long_views_7_days                        2402
v_long_views_30_days                       2402
'''

'\nv_cr_click_dislike_7_days                   980\nv_cr_click_dislike_1_days                   980\nv_cr_click_dislike_30_days                  980\n\n\nv_avg_watchtime_1_day                    220489\nv_avg_watchtime_7_day                    220489\nv_avg_watchtime_30_day                   220489\n\n\nv_frac_avg_watchtime_1_day_duration      401816\nv_frac_avg_watchtime_7_day_duration      401816\nv_frac_avg_watchtime_30_day_duration     401816\n\n\nv_month_views                              3284\nv_week_views                               3284\nv_day_views                                3284\n\n\nv_cr_click_like_1_days                     4869\nv_cr_click_like_30_days                    4869\nv_cr_click_like_7_days                     4869\n\n\nv_cr_click_comment_7_days                  1694\nv_cr_click_comment_30_days                 1694\nv_cr_click_comment_1_days                  1694\n\n\nv_cr_click_long_view_30_days              28910\nv_cr_click_long_view_1_days               

In [21]:
'''
Мусор фичи 
v_is_deleted
v_is_hidden
row_number
'''

'\nМусор фичи \nv_is_deleted\nv_is_hidden\nrow_number\n'

In [22]:
'''
Взять title и получить эмбеды 
Всякие лаг фичи, фурье фичи, по таймстемпу, цикличные фичи а также бинарные фичи по типу всякие праздники по типу нг 
по id создателей сделать агреггированные фичи 

Для видео продолжительностью более 5 минут: просмотр считается долгим, если пользователь посмотрел больше 25% длительности видео.
Для видео менее или равные 5 минут: просмотр считается долгим, если пользователь посмотрел более 30 секунд.

Разница между event_timestamp и v_pub_timestamp

Надо попробовать что Даня сказал с SSL и вот сделать тут также с текстом!
'''

'\nВзять title и получить эмбеды \nВсякие лаг фичи, фурье фичи, по таймстемпу, цикличные фичи а также бинарные фичи по типу всякие праздники по типу нг \nпо id создателей сделать агреггированные фичи \n\nДля видео продолжительностью более 5 минут: просмотр считается долгим, если пользователь посмотрел больше 25% длительности видео.\nДля видео менее или равные 5 минут: просмотр считается долгим, если пользователь посмотрел более 30 секунд.\n\nРазница между event_timestamp и v_pub_timestamp\n\nНадо попробовать что Даня сказал с SSL и вот сделать тут также с текстом!\n'

In [23]:
'''
Хорошие фичи

v_duration                               0.004272
v_likes                                  0.001689
'''

'\nХорошие фичи\n\nv_duration                               0.004272\nv_likes                                  0.001689\n'

In [24]:
'''
Тест и трейн один и тот же день. Надо об этом учитывать и делать фичи именно сильно дающие инфу о 2024-08-10 
А также получается надо минутные цикличные или часовые(Надо построить спец график(есть имя ему) )
'''

'\nТест и трейн один и тот же день. Надо об этом учитывать и делать фичи именно сильно дающие инфу о 2024-08-10 \nА также получается надо минутные цикличные или часовые(Надо построить спец график(есть имя ему) )\n'

In [25]:
#Регрессия или классификация? Склоняюсь к регрессии и потом просто переводе в нули и единички 

In [26]:
import numpy as np
import pandas as pd

# =========================
# Конфиг по умолчанию
# =========================
TZ = "Europe/Moscow"
DEFAULT_TS_UNIT = "s"      # 's' или 'ms' для event_timestamp
PUB_NAIVE_TZ = "UTC"       # если v_pub_datetime наивный, считаем его изначально в UTC

# =========================
# 1) Приведение времени
# =========================
def standardize_time(
    df: pd.DataFrame,
    ts_col: str = "event_timestamp",
    pub_col: str = "v_pub_datetime",
    ts_unit: str = DEFAULT_TS_UNIT,
    tz: str = TZ,
    pub_naive_tz: str = PUB_NAIVE_TZ,
) -> pd.DataFrame:
    """Возвращает копию с tz-aware датами: event_dt_msk, pub_dt_msk."""
    out = df.copy()

    # event_timestamp (числовой epoch)
    evt = pd.to_datetime(out[ts_col], unit=ts_unit, utc=True, errors="coerce").dt.tz_convert(tz)

    # v_pub_datetime (может быть tz-naive)
    pub = pd.to_datetime(out[pub_col], errors="coerce")
    if getattr(pub.dt, "tz", None) is None:
        pub = pub.dt.tz_localize(pub_naive_tz, nonexistent="shift_forward", ambiguous="NaT")
    pub = pub.dt.tz_convert(tz)

    out["event_dt_msk"] = evt
    out["pub_dt_msk"] = pub
    return out

# =========================
# 2) Интра-дневные фичи
# =========================
def add_intraday_features(
    df: pd.DataFrame,
    dt_col: str = "event_dt_msk",
    add_fourier: bool = True,
    K_HOUR: int = 2
) -> pd.DataFrame:
    """Сек/мин/час дня, циклические фичи по часу, прайм-тайм флаги."""
    out = df.copy()
    evt = out[dt_col]

    out["hour"] = evt.dt.hour.astype("int8")
    out["minute"] = evt.dt.minute.astype("int8")
    out["second"] = evt.dt.second.astype("int8")

    out["minute_of_day"] = (out["hour"] * 60 + out["minute"]).astype("int16")
    out["sec_of_day"] = ((out["hour"] * 3600) + (out["minute"] * 60) + out["second"]).astype("int32")

    # прайм-тайм (примерные окна — можно тюнить)
    sod = out["sec_of_day"]
    out["is_morning"] = sod.between(6*3600, 11*3600).astype("int8")
    out["is_day"]     = sod.between(11*3600, 17*3600).astype("int8")
    out["is_evening"] = sod.between(17*3600, 23*3600).astype("int8")
    out["is_night"]   = ((sod < 6*3600) | (sod >= 23*3600)).astype("int8")

    if add_fourier:
        # плавная доля суток [0,1)
        frac_day = (out["sec_of_day"].astype("float32") / 86400.0).clip(0, 1)
        two_pi = 2.0 * np.pi
        for k in range(1, K_HOUR + 1):
            ang = two_pi * k * frac_day
            out[f"f_day_sin_{k}"] = np.sin(ang).astype("float32")
            out[f"f_day_cos_{k}"] = np.cos(ang).astype("float32")

    return out

# =========================
# 3) Возраст от публикации
# =========================
def add_age_features(
    df: pd.DataFrame,
    evt_col: str = "event_dt_msk",
    pub_col: str = "pub_dt_msk",
) -> pd.DataFrame:
    """Возраст: секунды/часы/дни, лог-скейл, флаг отрицательной разницы."""
    out = df.copy()
    age_sec = (out[evt_col] - out[pub_col]).dt.total_seconds()
    out["age_sec"] = age_sec.astype("float32")
    out["age_hours"] = (out["age_sec"] / 3600.0).astype("float32")
    out["age_days"] = (out["age_sec"] / 86400.0).astype("float32")
    out["age_log1p"] = np.log1p(np.clip(out["age_sec"], a_min=0, a_max=None)).astype("float32")
    out["age_neg_flag"] = (out["age_sec"] < 0).astype("int8")

    # быстрые пороги свежести
    out["is_fresh_1h"]  = (out["age_sec"] <= 3600).astype("int8")
    out["is_fresh_3h"]  = (out["age_sec"] <= 3*3600).astype("int8")
    out["is_fresh_12h"] = (out["age_sec"] <= 12*3600).astype("int8")
    return out

# =========================
# 4) Квантили-бины возраста (из train) + маппинг в обе выборки
# =========================
def fit_age_bins(train_age_sec: pd.Series, n_bins: int = 10) -> np.ndarray:
    """Квантильные бины возраста (секунды), устойчивы к хвостам."""
    x = train_age_sec.dropna().clip(lower=0)
    qs = np.linspace(0, 1, n_bins + 1)
    bins = np.unique(np.quantile(x.to_numpy(), qs))
    if bins.size < 2:
        bins = np.array([0.0, float(x.max() if len(x) else 1.0)], dtype=float)
    return bins

def apply_age_bins(df: pd.DataFrame, bins: np.ndarray, col: str = "age_sec") -> pd.DataFrame:
    out = df.copy()
    out["age_bin"] = pd.cut(out[col].clip(lower=0), bins=bins, include_lowest=True)
    # числовой индекс бина (+ компактный int)
    out["age_bin_idx"] = out["age_bin"].cat.codes.astype("int16")
    return out

# =========================
# 5) Временные приоры (смутые CTR) по train + маппинг
# =========================
def _smoothed_ctr(sum_, cnt, alpha=2.0, beta=2.0):
    return (sum_ + alpha) / (cnt + alpha + beta)

def build_time_priors(
    train_df: pd.DataFrame,
    y_col: str = "y_long",
    alpha: float = 2.0,
    beta: float = 2.0,
) -> dict:
    """Считает p(y=1) по часу, 5-мин окнам, минуте дня и бинaм возраста на TRAIN."""
    p_glob = float(train_df[y_col].mean())

    g = train_df.groupby("hour")[y_col].agg(["sum", "count"])
    p_hour = _smoothed_ctr(g["sum"], g["count"], alpha, beta).rename("p_long_hour")

    g = train_df.groupby("minute_of_day")[y_col].agg(["sum", "count"])
    p_minute = _smoothed_ctr(g["sum"], g["count"], alpha, beta).rename("p_long_minute")

    g = (train_df.assign(min5_bucket=(train_df["minute_of_day"] // 5).astype("int16"))
                 .groupby("min5_bucket")[y_col].agg(["sum", "count"]))
    p_min5 = _smoothed_ctr(g["sum"], g["count"], alpha, beta).rename("p_long_min5")

    # по бинaм возраста (тут уже должен быть age_bin)
    g = train_df.groupby("age_bin")[y_col].agg(["sum", "count"])
    p_agebin = _smoothed_ctr(g["sum"], g["count"], alpha, beta).rename("p_long_agebin")
    # индекс лучше строкой для однозначного маппинга
    p_agebin.index = p_agebin.index.astype(str)

    return {
        "p_glob": p_glob,
        "p_hour": p_hour,
        "p_minute": p_minute,
        "p_min5": p_min5,
        "p_agebin": p_agebin,
    }

def apply_time_priors(df: pd.DataFrame, priors: dict) -> pd.DataFrame:
    out = df.copy()
    out["min5_bucket"] = (out["minute_of_day"] // 5).astype("int16")

    out["p_long_hour"]   = out["hour"].map(priors["p_hour"]).astype("float32")
    out["p_long_minute"] = out["minute_of_day"].map(priors["p_minute"]).astype("float32")
    out["p_long_min5"]   = out["min5_bucket"].map(priors["p_min5"]).astype("float32")

    # маппинг по строковому представлению age_bin
    agebin_str = out["age_bin"].astype(str)
    out["p_long_agebin"] = agebin_str.map(priors["p_agebin"]).astype("float32")

    # бэкоф глобальным средним
    for c in ["p_long_hour", "p_long_minute", "p_long_min5", "p_long_agebin"]:
        out[c] = out[c].fillna(priors["p_glob"]).astype("float32")
    return out

# =========================
# 6) Приор (video_id, hour) из TRAIN + бэкоф
# =========================
def add_video_hour_prior(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    y_col: str = "y_long",
    min_count: int = 100,
    alpha: float = 2.0,
    beta: float = 2.0,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    grp = train_df.groupby(["video_id", "hour"])[y_col].agg(["sum", "count"])
    keep = grp["count"] >= min_count
    grp = grp[keep]
    p_vh = _smoothed_ctr(grp["sum"], grp["count"], alpha, beta).rename("p_long_video_hour")

    for df in (train_df, test_df):
        key = df.set_index(["video_id", "hour"]).index
        df["p_long_video_hour"] = pd.Series(key.map(p_vh), index=df.index).astype("float32")
        # бэкоф к часовому приору (уже должен быть)
        df["p_long_video_hour"] = df["p_long_video_hour"].fillna(df["p_long_hour"]).astype("float32")
    return train_df, test_df

# =========================
# 7) Роллинги «в прошлое» (без утечки)
# =========================
def add_past_gap_features(
    df: pd.DataFrame,
    evt_col: str = "event_dt_msk",
    by_user: bool = True,
    by_uv: bool = True,
) -> pd.DataFrame:
    """
    time_since_prev_user: время с прошлого события пользователя
    time_since_prev_uv:   время с прошлого события той же пары (user,video)
    """
    out = df.copy().sort_values(evt_col).reset_index(drop=True)
    sec = lambda x: x.dt.total_seconds().astype("float32")

    if by_user and "user_id" in out.columns:
        prev_u = (
            out.groupby("user_id")[evt_col].shift(1)
        )
        out["time_since_prev_user"] = sec(out[evt_col] - prev_u).fillna(-1.0)

    if by_uv and {"user_id","video_id"}.issubset(out.columns):
        prev_uv = (
            out.groupby(["user_id","video_id"])[evt_col].shift(1)
        )
        out["time_since_prev_uv"] = sec(out[evt_col] - prev_uv).fillna(-1.0)

    return out

# =========================
# 8) Оркестратор: строим «с нуля»
# =========================
def build_temporal_features(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    ts_unit: str = DEFAULT_TS_UNIT,
    tz: str = TZ,
    pub_naive_tz: str = PUB_NAIVE_TZ,
    y_col: str = "y_long",
    n_age_bins: int = 10,
    alpha: float = 2.0,
    beta: float = 2.0,
    min_count_vh: int = 100,
) -> tuple[pd.DataFrame, pd.DataFrame, dict]:
    # 1) приведение времени
    tr = standardize_time(train_df, ts_unit=ts_unit, tz=tz, pub_naive_tz=pub_naive_tz)
    te = standardize_time(test_df,  ts_unit=ts_unit, tz=tz, pub_naive_tz=pub_naive_tz)

    # 2) базовые задачи времени
    tr = add_intraday_features(tr, add_fourier=True, K_HOUR=2)
    te = add_intraday_features(te, add_fourier=True, K_HOUR=2)

    # 3) возраст от публикации
    tr = add_age_features(tr)
    te = add_age_features(te)

    # 4) квантили-бины возраста (по train) + маппинг
    bins = fit_age_bins(tr["age_sec"], n_bins=n_age_bins)
    tr = apply_age_bins(tr, bins)
    te = apply_age_bins(te, bins)

    # 5) временные приоры из train и маппинг
    priors = build_time_priors(tr, y_col=y_col, alpha=alpha, beta=beta)
    tr = apply_time_priors(tr, priors)
    te = apply_time_priors(te, priors)

    # 6) (video,hour) приор с бэкофом
    tr, te = add_video_hour_prior(tr, te, y_col=y_col, min_count=min_count_vh, alpha=alpha, beta=beta)

    # 7) роллинги "в прошлое" без утечки (отдельно в каждом df)
    tr = add_past_gap_features(tr)
    te = add_past_gap_features(te)

    artifacts = {"age_bins": bins, "priors": priors}
    return tr, te, artifacts


In [27]:
# Пример: у нас уже есть train_full_new1 (с y_long) и test_full
train_time, test_time, artifacts = build_temporal_features(
    train_full_new1, 
    test_full_new1, 
    ts_unit="s",           # у тебя именно секунды
    tz="Europe/Moscow",
    pub_naive_tz="UTC",
    y_col="y_long",
    n_age_bins=12,         # можно 8-16; подберёшь по CV
    alpha=2.0, beta=2.0,
    min_count_vh=100       # приор (video,hour) только для достаточно частых
)

print(train_time.shape, test_time.shape)
train_time.filter(regex="^(sec_of_day|f_day_|age_|p_long_|time_since_)").head()


/tmp/ipykernel_37/2612827446.py:144: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = train_df.groupby("age_bin")[y_col].agg(["sum", "count"])


(3263353, 63) (1334132, 60)


,sec_of_day,f_day_sin_1,f_day_cos_1,f_day_sin_2,f_day_cos_2,age_sec,age_hours,age_days,age_log1p,age_neg_flag,age_bin,age_bin_idx,p_long_hour,p_long_minute,p_long_min5,p_long_agebin,p_long_video_hour,time_since_prev_user,time_since_prev_uv
0,0,0.0,1.0,0.0,1.0,13265531.0,3684.869629,153.536240,16.400681,0,"(13179702.0, 13304546.0]",3,0.48226,0.48935,0.498806,0.055709,0.48226,-1.0,-1.0
1,0,0.0,1.0,0.0,1.0,39946360.0,11096.210938,462.342133,17.503048,0,"(38012924.0, 50872940.0]",8,0.48226,0.48935,0.498806,0.646778,0.48226,-1.0,-1.0
2,0,0.0,1.0,0.0,1.0,13319931.0,3699.980713,154.165863,16.404772,0,"(13304546.0, 13759221.0]",4,0.48226,0.48935,0.498806,0.075433,0.48226,-1.0,-1.0
3,0,0.0,1.0,0.0,1.0,28715980.0,7976.661133,332.360870,17.172964,0,"(26606688.0, 38012924.0]",7,0.48226,0.48935,0.498806,0.584863,0.48226,-1.0,-1.0
4,0,0.0,1.0,0.0,1.0,35114164.0,9753.934570,406.413940,17.374115,0,"(26606688.0, 38012924.0]",7,0.48226,0.48935,0.498806,0.584863,0.48226,-1.0,-1.0


In [28]:
import re
import numpy as np
import pandas as pd
'''
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

# -------------------------
# 1) НОРМАЛИЗАЦИЯ ТЕКСТА
# -------------------------
def normalize_title_series(s: pd.Series) -> pd.Series:
    s = s.fillna("")
    # нижний регистр + единичные пробелы
    s = s.str.lower()
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

# -------------------------
# 2) РЕГЕКСЫ И СЛОВАРИ
# -------------------------
RX_EMOJI = re.compile(
    "["                                   # очень грубый детектор эмодзи
    "\U0001F300-\U0001F64F"
    "\U0001F680-\U0001F6FF"
    "\U0001F900-\U0001F9FF"
    "\u2600-\u26FF\u2700-\u27BF"
    "]+"
)

# ключевые хинты (EN/RU)
HINT_LONG  = [
    r"стрим", r"прямой эфир", r"запись стрима", r"трансляц",
    r"live", r"stream", r"podcast", r"подкаст", r"full episode", r"full version",
    r"полная верс", r"longplay", r"полный выпуск"
]
HINT_SHORT = [
    r"shorts?", r"\bshort\b", r"тизер", r"teaser", r"трейлер", r"trailer",
    r"clip\b", r"клип\b", r"highlights?", r"нарезка", r"cut\b", r"шортс"
]
HINT_EDU   = [r"\bкак\b", r"how to", r"tutorial", r"урок", r"гайд", r"инструкц", r"обзор", r"review"]
HINT_OFFICIAL = [r"official", r"официальн"]

# part/episode/season
RX_PART_EP = re.compile(r"(?:\bpart\s*\d+\b|\bчаст[ьи]\s*\d+\b|\bs\d{1,2}e\d{1,2}\b)", re.I)

# годы/числа
RX_YEAR = re.compile(r"\b(19\d{2}|20\d{2}|210\d)\b")

# паттерны длительности внутри title (грубо)
# поддерживаем: "10 часов", "2 часа 30 минут", "1h 20m", "90 мин", "00:30", "1:20:00"
RX_TIME_BLOCKS = [
    (re.compile(r"(\d+)\s*(?:час|часа|часов|h|hr|hour)s?\b", re.I), 60),           # часы → минуты
    (re.compile(r"(\d+)\s*(?:мин|минут|minute|m)\b", re.I), 1),                   # минуты
    (re.compile(r"(\d+)\s*(?:сек|секунд|second|s)\b", re.I), 1/60),               # секунды → минуты
    (re.compile(r"\b(\d{1,2}):([0-5]\d)(?::([0-5]\d))?\b"), "hh:mm:ss"),          # 1:20 или 1:20:00
]

def _extract_title_duration_min(title: str) -> float:
    """Пытаемся вытащить упоминание длительности из title, вернуть минуты (float) или np.nan."""
    if not title:
        return np.nan
    t = title

    # форматы типа 1:20:00 / 1:20
    m = RX_TIME_BLOCKS[3][0].search(t)
    if m:
        h = int(m.group(1))
        mm = int(m.group(2))
        ss = int(m.group(3)) if m.group(3) else 0
        return h*60 + mm + ss/60.0

    total_min = 0.0
    found = False
    for rx, mul in RX_TIME_BLOCKS[:3]:
        for g in rx.findall(t):
            try:
                num = float(g if isinstance(g, str) else g[0])
            except Exception:
                continue
            total_min += num * (mul if isinstance(mul, (int, float)) else 1.0)
            found = True
    return total_min if found else np.nan

# -------------------------
# 3) ЛЕКСИКО-СТРУКТУРНЫЕ ФИЧИ
# -------------------------
def extract_title_lex_features(df: pd.DataFrame, title_col: str = "title") -> pd.DataFrame:
    out = df.copy()
    t = normalize_title_series(out[title_col])

    # базовые длины
    out["title_len_char"] = t.str.len().astype("int32")
    out["title_n_words"]  = t.str.split().str.len().fillna(0).astype("int16")

    # доли символов
    # кириллица/латиница/цифры/верхний регистр
    s = out[title_col].fillna("")
    has_cyr = s.str.contains(r"[А-Яа-яЁё]")
    has_lat = s.str.contains(r"[A-Za-z]")
    out["title_has_cyr"] = has_cyr.astype("int8")
    out["title_has_lat"] = has_lat.astype("int8")

    # грубая доля цифр/знаков/прописных
    out["title_digits_ratio"] = s.str.count(r"\d").div(out["title_len_char"].replace(0, np.nan)).fillna(0).astype("float32")
    out["title_punct_ratio"]  = s.str.count(r"[^\w\s]").div(out["title_len_char"].replace(0, np.nan)).fillna(0).astype("float32")
    out["title_excl_cnt"]     = s.str.count("!").astype("int16")
    out["title_quest_cnt"]    = s.str.count("\?").astype("int16")
    out["title_hash_cnt"]     = s.str.count("#").astype("int16")
    out["title_emoji_cnt"]    = s.apply(lambda x: len(RX_EMOJI.findall(x))).astype("int16")

    # ключевые хинты (binary)
    def any_match(patterns, text):
        for p in patterns:
            if re.search(p, text, re.I):
                return 1
        return 0

    out["title_hint_long"]   = t.apply(lambda x: any_match(HINT_LONG, x)).astype("int8")
    out["title_hint_short"]  = t.apply(lambda x: any_match(HINT_SHORT, x)).astype("int8")
    out["title_hint_edu"]    = t.apply(lambda x: any_match(HINT_EDU, x)).astype("int8")
    out["title_hint_offcl"]  = t.apply(lambda x: any_match(HINT_OFFICIAL, x)).astype("int8")

    # part/episode + год
    out["title_has_part_ep"] = t.str.contains(RX_PART_EP).astype("int8")
    out["title_has_year"]    = t.str.contains(RX_YEAR).astype("int8")

    # длительность, упомянутая в title
    out["title_dur_min_inferred"] = t.apply(_extract_title_duration_min).astype("float32")
    out["title_has_dur"]          = out["title_dur_min_inferred"].notna().astype("int8")

    # полезные пересечения
    out["title_is_trailer_or_teaser"] = ((out["title_hint_short"]==1) & (t.str.contains(r"trailer|тизер|teaser", regex=True))).astype("int8")
    out["title_is_stream_live"]       = out["title_hint_long"]
    out["title_is_shorts"]            = t.str.contains(r"\bshorts?\b|шортс", regex=True).astype("int8")

    # длиновые бины
    out["title_len_char_bin"] = pd.cut(out["title_len_char"], bins=[-1, 20, 35, 60, 100, 1000], labels=[0,1,2,3,4]).astype("int8")
    out["title_n_words_bin"]  = pd.cut(out["title_n_words"],  bins=[-1, 3, 6, 12, 20, 100],   labels=[0,1,2,3,4]).astype("int8")

    return out

# -------------------------
# 4) TF-IDF (char n-grams) → SVD
# -------------------------
def add_tfidf_svd_features(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    title_col: str = "title",
    max_features: int = 30000,
    n_components: int = 32,
    ngram_range=(3,5),
    random_state: int = 42,
):
    tr = train_df.copy()
    te = test_df.copy()

    tr_text = normalize_title_series(tr[title_col]).fillna("")
    te_text = normalize_title_series(te[title_col]).fillna("")

    vec = TfidfVectorizer(
        analyzer="char",
        ngram_range=ngram_range,
        max_features=max_features,
        min_df=2
    )
    X_tr = vec.fit_transform(tr_text)
    X_te = vec.transform(te_text)

    svd = TruncatedSVD(n_components=n_components, random_state=random_state)
    Z_tr = svd.fit_transform(X_tr).astype("float32")
    Z_te = svd.transform(X_te).astype("float32")

    for i in range(n_components):
        tr[f"title_svd_{i:02d}"] = Z_tr[:, i]
        te[f"title_svd_{i:02d}"] = Z_te[:, i]

    artifacts = {"vectorizer": vec, "svd": svd}
    return tr, te, artifacts

# -------------------------
# 5) КЛАСТЕРЫ + ПРИОРЫ ПО КЛАСТЕРАМ
# -------------------------
def add_title_cluster_priors(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    y_col: str = "y_long",
    n_clusters: int = 50,
    base_cols_pattern: str = r"^title_svd_\d+$",
    alpha: float = 2.0,
    beta: float = 2.0,
    random_state: int = 42,
):
    """Кластерим SVD-компоненты заголовков, считаем p(y=1) по кластерам (train) и мапим в обе выборки."""
    tr = train_df.copy()
    te = test_df.copy()

    svd_cols = [c for c in tr.columns if re.match(base_cols_pattern, c)]
    X_tr = tr[svd_cols].to_numpy(dtype=np.float32)
    X_te = te[svd_cols].to_numpy(dtype=np.float32)

    km = KMeans(n_clusters=n_clusters, random_state=random_state, n_init="auto")
    tr["_title_cluster"] = km.fit_predict(X_tr).astype("int16")
    te["_title_cluster"] = km.predict(X_te).astype("int16")

    # смутый CTR по кластерам
    g = tr.groupby("_title_cluster")[y_col].agg(["sum","count"])
    p = (g["sum"] + alpha) / (g["count"] + alpha + beta)
    p = p.rename("p_long_title_cluster")

    tr["p_long_title_cluster"] = tr["_title_cluster"].map(p).astype("float32")
    te["p_long_title_cluster"] = te["_title_cluster"].map(p).astype("float32")

    # необязательный feature: расстояние до центроида (инерция)
    centers = km.cluster_centers_.astype(np.float32)
    tr["title_cluster_dist"] = ((X_tr - centers[tr["_title_cluster"]])**2).sum(axis=1)**0.5
    te["title_cluster_dist"] = ((X_te - centers[te["_title_cluster"]])**2).sum(axis=1)**0.5
    tr["title_cluster_dist"] = tr["title_cluster_dist"].astype("float32")
    te["title_cluster_dist"] = te["title_cluster_dist"].astype("float32")

    artifacts = {"kmeans": km, "cluster_priors": p}
    return tr, te, artifacts

# -------------------------
# 6) ОРКЕСТРАТОР ДЛЯ TITLE
# -------------------------
def build_title_features(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    title_col: str = "title",
    y_col: str = "y_long",
    tfidf_max_features: int = 30000,
    svd_components: int = 32,
    kmeans_clusters: int = 50,
):
    # a) лексические/структурные
    tr = extract_title_lex_features(train_df, title_col=title_col)
    te = extract_title_lex_features(test_df,  title_col=title_col)

    # b) TF-IDF (char) → SVD
    tr, te, arts_tfidf = add_tfidf_svd_features(
        tr, te, title_col=title_col,
        max_features=tfidf_max_features, n_components=svd_components
    )

    # c) Кластеры заголовков + приор p(y=1) по кластерам (по train)
    tr, te, arts_cl = add_title_cluster_priors(
        tr, te, y_col=y_col, n_clusters=kmeans_clusters
    )

    artifacts = {"tfidf_svd": arts_tfidf, "title_clusters": arts_cl}
    return tr, te, artifacts

'''

'\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.decomposition import TruncatedSVD\nfrom sklearn.cluster import KMeans\n\n# -------------------------\n# 1) НОРМАЛИЗАЦИЯ ТЕКСТА\n# -------------------------\ndef normalize_title_series(s: pd.Series) -> pd.Series:\n    s = s.fillna("")\n    # нижний регистр + единичные пробелы\n    s = s.str.lower()\n    s = s.str.replace(r"\\s+", " ", regex=True).str.strip()\n    return s\n\n# -------------------------\n# 2) РЕГЕКСЫ И СЛОВАРИ\n# -------------------------\nRX_EMOJI = re.compile(\n    "["                                   # очень грубый детектор эмодзи\n    "🌀-🙏"\n    "🚀-\U0001f6ff"\n    "🤀-🧿"\n    "☀-⛿✀-➿"\n    "]+"\n)\n\n# ключевые хинты (EN/RU)\nHINT_LONG  = [\n    r"стрим", r"прямой эфир", r"запись стрима", r"трансляц",\n    r"live", r"stream", r"podcast", r"подкаст", r"full episode", r"full version",\n    r"полная верс", r"longplay", r"полный выпуск"\n]\nHINT_SHORT = [\n    r"shorts?", r"\x08short\x08", r"ти

In [29]:
# Предполагаем, что у тебя уже есть:
# - train_time (или train_full_new1_feats) с колонками ['title', 'y_long', ...]
# - test_time  (или test_full_feats)       с колонкой    ['title', ...]
# (названия можешь подставить свои)
'''
train_title, test_title, title_artifacts = build_title_features(
    train_time, test_time,
    title_col="title",
    y_col="y_long",
    tfidf_max_features=30000,   # можно 50k, если хватает памяти
    svd_components=32,          # 16–64 — здоровый диапазон
    kmeans_clusters=50          # 30–100 — попробуй подобрать по CV
)

# Быстрый sanity:
feat_cols = [c for c in train_title.columns if c.startswith(("title_", "p_long_title_cluster"))]
print(len(feat_cols), "title-features")
print(train_title[feat_cols].head())
'''

'\ntrain_title, test_title, title_artifacts = build_title_features(\n    train_time, test_time,\n    title_col="title",\n    y_col="y_long",\n    tfidf_max_features=30000,   # можно 50k, если хватает памяти\n    svd_components=32,          # 16–64 — здоровый диапазон\n    kmeans_clusters=50          # 30–100 — попробуй подобрать по CV\n)\n\n# Быстрый sanity:\nfeat_cols = [c for c in train_title.columns if c.startswith(("title_", "p_long_title_cluster"))]\nprint(len(feat_cols), "title-features")\nprint(train_title[feat_cols].head())\n'

# SKF catboost

In [30]:
useless=['v_cr_click_dislike_7_days','v_cr_click_dislike_30_days',
         'v_avg_watchtime_7_day','v_avg_watchtime_30_day',
         'v_frac_avg_watchtime_7_day_duration','v_frac_avg_watchtime_30_day_duration',
         'v_month_views','v_week_views',
         'v_cr_click_like_7_days','v_cr_click_like_30_days',
         'v_cr_click_comment_7_days','v_cr_click_comment_30_days',
         'v_cr_click_long_view_7_days','v_cr_click_long_view_30_days',
         'v_category_popularity_percent_30_days',
         'v_long_views_7_days','v_long_views_30_days',
         'v_is_deleted','v_is_hidden','row_number',
         
         'user_id','category_id','author_id', 'video_id',
         'description','title',
         'region','city']

In [31]:
#train_full_new1=train_full_new.drop(columns=useless)

#test_full_new1=test_full.drop(columns=useless)

In [32]:
#train_new

In [33]:
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd

def make_long_watch_flag(df, duration_col='v_duration', watch_col='watchtime', strict=True, unit='s'):
    # 1) to seconds (if needed) and numeric
    dur = pd.to_numeric(df[duration_col], errors='coerce')
    wt  = pd.to_numeric(df[watch_col],  errors='coerce')
    if unit == 'ms':
        dur = dur / 1000.0
        wt  = wt  / 1000.0

    # 2) thresholds
    thr = np.where(dur > 300.0, dur * 0.25, 30.0)  # 300s = 5 min

    # 3) validity mask and comparison
    valid = dur.notna() & wt.notna() & (dur > 0) & (wt >= 0)
    cmp = (wt > thr) if strict else (wt >= thr)

    # 4) build nullable Int8 target safely (pandas Series + pd.NA)
    y = pd.Series(cmp, index=df.index)
    y.loc[~valid] = pd.NA
    df['y_long'] = y.astype('Int8')  # 0/1 with NA

    # optional: keep threshold for debugging
    df['longwatch_threshold_sec'] = pd.Series(np.where(valid, thr, np.nan), index=df.index)

    return df


# Example:
#train_full_new1 = make_long_watch_flag(train_full_new1, 'v_duration', 'watchtime', strict=True)
# If test has watchtime and you want the same flag there:
# test = make_long_watch_flag(test, 'v_duration', 'watchtime', strict=True)

# Quick sanity check
#print(train_full_new1['y_long'].value_counts(dropna=True))


In [34]:
train_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263353 entries, 0 to 3263352
Data columns (total 63 columns):
 #   Column                                Dtype                        
---  ------                                -----                        
 0   event_timestamp                       datetime64[ms, Europe/Moscow]
 1   user_id                               object                       
 2   region                                object                       
 3   city                                  object                       
 4   video_id                              object                       
 5   watchtime                             int64                        
 6   v_pub_datetime                        datetime64[ms]               
 7   v_total_comments                      uint64                       
 8   v_year_views                          uint64                       
 9   v_day_views                           uint64                       
 10  v_like

In [35]:
useless_cols=['event_dt_msk','pub_dt_msk','age_bin','event_timestamp',
          'v_pub_datetime']

In [36]:
train_time=train_time.drop(columns=useless_cols)

test_time=test_time.drop(columns=useless_cols)

In [37]:
train_time=train_time[:2_000_000]

In [38]:
X=train_time.drop(columns=['watchtime', 'y_long', 'longwatch_threshold_sec', 'title', 'description'])
y=train_time['y_long']


In [39]:
X_train, X_eval, y_train, y_eval=train_test_split(X, y, test_size=0.15, random_state=seed)

In [40]:
#for now just one catboost 

model=CatBoostClassifier(
    iterations=1000,
    depth=6,
    learning_rate=0.03,
    loss_function='Logloss',
    eval_metric='F1',
    task_type='GPU',
    #l2_leaf_reg=,
    
)

In [41]:
cat_feats=[ 'user_id','video_id','category_id','author_id',  'region','city']

In [42]:
model.fit(X_train, y_train, eval_set=(X_eval, y_eval),cat_features=cat_feats, verbose=200, early_stopping_rounds=500)

0:	learn: 0.7935981	test: 0.7943852	best: 0.7943852 (0)	total: 5.88s	remaining: 1h 37m 53s
200:	learn: 0.8136516	test: 0.8161151	best: 0.8161233 (196)	total: 29.5s	remaining: 1m 57s
400:	learn: 0.8176377	test: 0.8195704	best: 0.8195920 (399)	total: 52.6s	remaining: 1m 18s
600:	learn: 0.8215543	test: 0.8226410	best: 0.8226450 (599)	total: 1m 16s	remaining: 50.7s
800:	learn: 0.8246161	test: 0.8249453	best: 0.8249453 (800)	total: 1m 39s	remaining: 24.8s
999:	learn: 0.8266115	test: 0.8265104	best: 0.8265104 (999)	total: 2m 3s	remaining: 0us
bestTest = 0.826510408
bestIteration = 999


# Just mean of them 

In [43]:
num_feats=train_full_new1.select_dtypes(include='number').columns.tolist()


In [44]:
corr=train_full_new1[num_feats].corrwith(train_full_new1['y_long'])

In [45]:
corr

watchtime                               0.411520
v_total_comments                       -0.017482
v_year_views                           -0.011971
v_day_views                             0.050856
v_likes                                 0.010362
v_dislikes                             -0.001967
v_duration                             -0.070463
v_cr_click_vtop_7_days                  0.006475
v_cr_click_vtop_30_days                 0.006475
v_cr_click_like_1_days                 -0.003513
v_cr_click_dislike_1_days               0.000486
v_cr_click_vtop_1_days                  0.006475
v_cr_click_long_view_1_days             0.550106
v_cr_click_comment_1_days              -0.000820
v_avg_watchtime_1_day                   0.050975
v_frac_avg_watchtime_1_day_duration     0.007242
v_category_popularity_percent_7_days    0.098284
v_long_views_1_days                     0.166037
y_long                                  1.000000
longwatch_threshold_sec                -0.081352
dtype: float64

In [46]:
X_test=test_time.drop(columns=['title', 'description'])

In [47]:
preds=model.predict(X_test)

In [48]:
sample['target']=preds

In [49]:
sample.to_csv('submission.csv', index=False)

# Meta Models